Goodreads Recommender System
MINI PROJECT DESCRIPTION
Follow these instructions to complete the mini project.

# 1 - Download the Dataset
Method 1
Download the dataset from the following link:
https://www.kaggle.com/jealousleopard/goodreadsbooks/download

# 2  - Read the Dataset
Read the dataset into a Pandas Dataframe!

In [1]:
import pandas as pd
df = pd.read_csv("books.csv",error_bad_lines=False)
df.head()

/tmp/ipykernel_617/1405851289.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("books.csv",error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


# 3  -Popularity-based Recommender
Create a function named Popularity Recommender and use it to recommend books based on popularity.
Use a weighted rank similar to that used in the IMDB rating example in Lesson 2.

In [2]:
def PopularityRecommender(df):
    #Define the minimum vote count
    minimum_vote_count = 0.75 * df['ratings_count'].max()
    #Define C – the mean rating
    mean_rating = df['average_rating'].mean()

    #Calculate weighted_rating
    df['weighted_rating'] = ((df['ratings_count'] / (df['ratings_count'] + minimum_vote_count)) * df['average_rating']) + ((minimum_vote_count / (df['ratings_count'] + minimum_vote_count)) * mean_rating)

    #Return the top 5 recommendations
    recommendations = df.sort_values(by = 'weighted_rating', ascending = False).head(5)
    
    return recommendations
PopularityRecommender(df)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,weighted_rating
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,4.187089
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,4.174464
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,4.147771
4415,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.,4.128198
23,34,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,4.36,0618346252,9780618346257,eng,398,2128944,13670,9/5/2003,Houghton Mifflin Harcourt,4.096661


# 4 Content-based Recommender
Create a function named Content-based Recommender and use it to recommend books based on content.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Instantiate a new Vectorizer object 
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['title'] = df['title'].fillna('')

#This line transforms the description of the movies to the tfidf #matrix needed

tfidf_matrix = tfidf.fit_transform(df['title'])

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
distance_matrix = cosine_similarity(tfidf_matrix)

indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
Harry Potter and the Half-Blood Prince (Harry Potter  #6)           0
Harry Potter and the Order of the Phoenix (Harry Potter  #5)        1
Harry Potter and the Chamber of Secrets (Harry Potter  #2)          2
Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)         3
Harry Potter Boxed Set  Books 1-5 (Harry Potter  #1-5)              4
                                                                ...  
Expelled from Eden: A William T. Vollmann Reader                11118
You Bright and Risen Angels                                     11119
The Ice-Shirt (Seven Dreams #1)                                 11120
Poor People                                                     11121
Las aventuras de Tom Sawyer                                     11122
Length: 11123, dtype: int64

In [5]:
id_ = indices["Poor People"]

In [6]:
def ContentBasedRecommender(title):
        # Fetch the index of the movie itself
    id_ = indices[title]

# Create a list of tuples-Each tuple has an id and a distance to the original movie 
    distances = list(enumerate(distance_matrix[id_]))

    # Sort the movies based on the distances 
    distances = sorted(distances, key=lambda x: x[1], reverse = True)

    # Get the scores of the 5 most similar movies
    # Skip the first most similar item because it is the movie itself. 
    distances = distances[1:6]

    # Get the movie indices
    recommendations = [distance[0] for distance in distances]
    

    # Return the recommendations
    return df['title'].iloc[recommendations]
ContentBasedRecommender('Poor People')

7708                  The Book of Other People
3012    The Working Poor: Invisible in America
3435                            All New People
9510                       A Man of the People
217              We Were Not Like Other People
Name: title, dtype: object

# 5 - TF-IDF Vectorizer
Use TF-IDF Vectorizer on the author data for each book.

In [7]:
tfidf_matrix = tfidf.fit_transform(df['authors'])

# 6 - Distance matrix
Choose cosine similarity for pairwise distances comparison

In [8]:
distance_matrix = cosine_similarity(tfidf_matrix)